In [10]:
dic = {}
dic['us-south-central'] = '13.65.92.139'
dic['us-west1'] = "13.93.236.162"
dic['us-central'] = "localhost"
dic['asia-japan-east'] = "13.78.83.9"
dic['eu-north-europe'] = '52.178.201.184'
dic['eu-uk-west'] = '51.141.11.143'
dic['asia-japan-west'] = '104.214.146.200'
def get_percentile(file_name, percentile):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                if len(l) == 0 or len(l) == 1:
                    continue

                d[l[0]].append(int(l[1]))
    dic = {}
    import numpy as np
    for key, value in d.iteritems():
        a = np.array(value)
#         print np.amax(a)
        p = np.percentile(a, percentile)/1000
        dic[key] = p
    return dic

def data_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Put_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(fp + storage_name + "_storage_server.log", percentile)['Put_Azure_' + storage_name]
    difference = thrift_latency - azure_latency
    return '{:10} {:10} {:10}'.format(str(thrift_latency), str(azure_latency), str(difference))
    
def metadata_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Accept_Phase_thrfit_Table_' + storage_name]
    azure_latency = get_percentile(fp + storage_name + "_table_server.log", percentile)['Accept_Phase_AzureTable_' + storage_name]
    difference = thrift_latency - azure_latency
    return '{:10} {:10} {:10}'.format(str(thrift_latency), str(azure_latency), str(difference))

def get_data_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['GET_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(fp + storage_name + "_storage_server.log", percentile)['Get_Azure_' + storage_name]
    difference = thrift_latency - azure_latency
    return '{:10} {:10} {:10}'.format(str(thrift_latency), str(azure_latency), str(difference))
    
def get_metadata_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Get_Highest_Version_Thrift_Table_' + storage_name]
    azure_latency = get_percentile(fp + storage_name + "_table_server.log", percentile)['Get_Highest_Version_Number_AzureTable_' + storage_name]
    difference = thrift_latency - azure_latency
    return '{:10} {:10} {:10}'.format(str(thrift_latency), str(azure_latency), str(difference))


In [14]:
def put_result(fp, percentile, size, three_dc):
    f = fp + 'performance_log/giza_latency_put_' + size + '.txt.log'
    print ' '
    print '{:20} {:20}'.format('Put:', str(get_percentile(f, percentile)['Put']))
    print '{:20} {:20}'.format ('Data Path Put:', str(get_percentile(f, percentile)['SendErasureEncode']))
    for i in range(len(three_dc)):
        print '{:20} {:20}'.format(three_dc[i] + ' Put:', data_breakdown_row(fp, three_dc[i], percentile))
    print ' '
    get_version_latency = get_percentile(f, percentile)['GetLocalHighestCommittedVersionNumber']
    fast_paxos_latency = get_percentile(f, percentile)['DoPhase2']
    print '{:20} {:20}'.format('Metadata Path Put:', str(get_version_latency + fast_paxos_latency))
    print '{:20} {:20}'.format('Get Highest Version:', str(get_version_latency))
    print '{:20} {:20}'.format('Do Fast Paxos:', str(fast_paxos_latency))
    for i in range(len(three_dc)):
        print '{:20} {:20}'.format(three_dc[i] + ' Put:', metadata_breakdown_row(fp, three_dc[i], percentile))

def get_result(fp, percentile, size, three_dc):
    f = fp + 'performance_log/giza_latency_get_' + size + '.txt.log'
    print 'Get:           ' + str(get_percentile(f, percentile)['mainGet'])
    print 'Data Path Get: ' + str(get_percentile(f, percentile)['GetErasureEncode'])
    print three_dc[0] + '       Get: ' + get_data_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '      Get: ' + get_data_breakdown_row(fp, three_dc[1], percentile)
    print ' '
    get_version_latency = get_percentile(f, percentile)['GetLocal']
    print 'Get Local Highest Version: ' + str(get_version_latency)
    print 'Get Global Highest Version: ' + str(get_percentile(f, percentile)['GetGlobalHighestCommittedVersion'])
    print three_dc[0] + '        Get: ' + get_metadata_breakdown_row(fp, three_dc[0], percentile)
    print three_dc[1] + '       Get: ' + get_metadata_breakdown_row(fp, three_dc[1], percentile)


In [22]:
# 2-1 US 4mb
three_dc = ['us-central', 'eu-north-europe', 'asia-japan-east']
sizes = ['256kb']#, '1mb', '2mb', '4mb']
percentiles = [10, 50, 95]
task = '2-1 US'
for p in percentiles:
    print 'PERCENTILES: ' + str(p)
    metadata = []
    metadata_2rtt = []
    data = []
    latency = []
    for s in sizes:
        print s 
        fp = s + '/put/'
        f = fp + 'performance_log/giza_latency_put_' + s + '.txt.log'
#         plot.append(get_percentile(f, p))['Put']
        put_result(fp, p, s, three_dc)
        print '--------------------------------------------------------'
    
# put_result('4mb/put/', 50, '4mb', three_dc)
# print '###########################################################'
# put_result('2mb/put/', 50, '2mb', three_dc)
# print '###########################################################'
# put_result('1mb/put/', 50, '1mb', three_dc)
# print '###########################################################'
# put_result('256kb/put/', 50, '256kb', three_dc)
# print '###########################################################'

PERCENTILES: 10
256kb
 
Put:                 205.0244            
Data Path Put:       155.119             
us-central Put:      12.8271    12.3958    0.4313    
eu-north-europe Put: 123.657    17.9876    105.6694  
asia-japan-east Put: 152.8627   9.3786     143.4841  
 
Metadata Path Put:   203.0593            
Get Highest Version: 5.8178              
Do Fast Paxos:       197.2415            
us-central Put:      60.864     60.7707    0.0933    
eu-north-europe Put: 164.8839   62.7139    102.17    
asia-japan-east Put: 196.7793   57.5849    139.1944  
--------------------------------------------------------
PERCENTILES: 50
256kb
 
Put:                 211.544             
Data Path Put:       157.1395            
us-central Put:      14.2515    13.787     0.4645    
eu-north-europe Put: 126.619    20.2825    106.3365  
asia-japan-east Put: 154.3815   10.348     144.0335  
 
Metadata Path Put:   209.5885            
Get Highest Version: 6.7795              
Do Fast Paxos:       202.80

In [24]:
three_dc = ['central', 'north-europe', 'japan-east']
put_result('2-1-world/4mb/put/', 95, '4mb', three_dc)

IOError: [Errno 2] No such file or directory: '2-1-world/4mb/put/performance_log/giza_latency_put_4mb.txt.log'

In [4]:
put_result('2-1-us/1mb/put/', 95, '1mb')
print '###########################################################'
get_result('2-1-us/1mb/get/', 95, '1mb')

Put              : 217.65365
Data Path     Put: 158.1349
Central       Put: 43.73165  |  42.6922  |  1.03945
SCentral      Put: 134.3779  |  101.714  |  32.6639
West1         Put: 136.00105  |  87.006  |  48.99505
----------------------------------------------------------
Metadata Path Put: 207.2153
Get Highest Version: 15.1944
Do Fast Paxos     : 192.0209
Central        Put: 78.16415  |  78.02175  |  0.1424
SCentral       Put: 155.2838  |  131.5869  |  23.6969
West1          Put: 175.1092  |  137.04665  |  38.06255
###########################################################
Get:               168.74035
Data Path     Get: 139.17215
Central       Get: 61.58115  |  60.9419  |  0.63925
SCentral      Get: 129.24795  |  103.221  |  26.02695
----------------------------------------------------------
Get Local Highest Version: 31.73155
Get Global Highest Version: 57.9474
Central        Get: 12.05605  |  11.96315  |  0.0929
SCentral       Get: 56.6965  |  32.708  |  23.9885


In [5]:
put_result('2-1-us/256kb/put/', 95, '256kb')
print '###########################################################'
get_result('2-1-us/256kb/get/', 95, '256kb')

Put              : 218.9031
Data Path     Put: 125.9818
Central       Put: 28.93765  |  28.4837  |  0.45395
SCentral      Put: 104.152  |  74.0681  |  30.0839
West1         Put: 111.3456  |  66.4388  |  44.9068
----------------------------------------------------------
Metadata Path Put: 212.9071
Get Highest Version: 15.5203
Do Fast Paxos     : 197.3868
Central        Put: 26.9176  |  26.816  |  0.1016
SCentral       Put: 162.94365  |  139.68505  |  23.2586
West1          Put: 187.01905  |  149.18755  |  37.8315
###########################################################
Get:               171.8258
Data Path     Get: 140.0561
Central       Get: 53.73035  |  53.5461  |  0.18425
SCentral      Get: 134.60735  |  110.2424  |  24.36495
----------------------------------------------------------
Get Local Highest Version: 30.5646
Get Global Highest Version: 59.13075
Central        Get: 13.2561  |  13.1763  |  0.0798
SCentral       Get: 51.88945  |  28.5877  |  23.30175


In [9]:
def get_data_breakdown_row(storage_name, percentile):
    thrift_latency = get_percentile("giza_trace.log", percentile)['GET_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(storage_name + "_storage_server.log", percentile)['Get_Azure_us-' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)
    
def get_metadata_breakdown_row(storage_name, percentile):
    thrift_latency = get_percentile("giza_trace.log", percentile)['Get_Highest_Version_Thrift_Table_us-' + storage_name]
    azure_latency = get_percentile(storage_name + "_table_server.log", percentile)['Get_Highest_Version_Number_AzureTable_us-' + storage_name]
    difference = thrift_latency - azure_latency
    return str(thrift_latency) + "  |  " + str(azure_latency) + "  |  " + str(difference)

percentile = 95
size = '4mb'
f = 'performance_log/giza_latency_get_' + size + '.txt.log'
print 'Get:               ' + str(get_percentile(f, percentile)['mainGet'])
print 'Data Path     Get: ' + str(get_percentile(f, percentile)['GetErasureEncode'])
print 'Central       Get: ' + data_breakdown_row('central', percentile)
print 'SCentral      Get: ' + data_breakdown_row('south-central', percentile)
print '----------------------------------------------------------'
get_version_latency = get_percentile(f, percentile)['GetLocal']
print 'Get Local Highest Version: ' + str(get_version_latency)
print 'Get Global Highest Version: ' + str(get_percentile(f, percentile)['GetGlobalHighestCommittedVersion'])
print 'Central        Get: ' + metadata_breakdown_row('central', percentile)
print 'SCentral       Get: ' + metadata_breakdown_row('south-central', percentile)

Get:               677.275
Data Path     Get: 550.452


KeyError: 'Put_Thrift_Storage_localhost'

In [ ]:
def get_std(file_name, percentile):
    from collections import defaultdict
    d = defaultdict(list)
    with open(file_name) as f:
        for line in f:
            l = line.split()
            if len(l) == 3:
                key = l[0] + l[1]
                d[key].append(int(l[2]))
            elif len(l) == 4:
                key = l[0] + l[1] + l[2]
                d[key].append(int(l[3]))
            else:
                d[l[0]].append(int(l[1]))
    dic = {}
    import numpy as np
    for key, value in d.iteritems():
        a = np.array(value)
#         print np.amax(a)
        p = np.std(a, percentile)/1000
        dic[key] = p
    return dic

In [23]:
def data_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Put_Thrift_Storage_' + dic[storage_name]]
    azure_latency = get_percentile(fp + storage_name + "_storage_server.log", percentile)['Put_Azure_' + storage_name]
    difference = thrift_latency - azure_latency
    return '{:10} {:10} {:10}'.format(str(thrift_latency), str(azure_latency), str(difference))
def metadata_breakdown_row(fp, storage_name, percentile):
    thrift_latency = get_percentile(fp + "giza_trace.log", percentile)['Accept_Phase_thrfit_Table_' + storage_name]
    azure_latency = get_percentile(fp + storage_name + "_table_server.log", percentile)['Accept_Phase_AzureTable_' + storage_name]
    difference = thrift_latency - azure_latency
    return azure_latency
#     return '{:10} {:10} {:10}'.format(str(thrift_latency), str(azure_latency), str(difference))


In [ ]:
metadata_breakdown_row()